In [2]:
from utils import read_data, simple_predictors, evaluation, rollingCV, viz, knn, random_forest, parameters, predictions
import pandas as pd
from IPython.display import display

In [3]:
RD = read_data.ReadData()
viz = viz.Viz()
Eval = evaluation.Evaluation()
hd = RD.read_Hd()
K = 8
RCV = rollingCV.RollingCV(hd[::-1], K, 30, 14, 30)
SimplePredictors = simple_predictors.SimplePredictors(hd)
KNN = knn.KNN(Eval, 60)
RandomForest = random_forest.RandomForest(Eval)

In [4]:
Params = parameters.Parameters()
Predictions = predictions.Predictions()
points_of_sales = sorted(hd.id.unique())

In [9]:
display(RCV.getKthSet(1, isML=True)[0])

,dzien_rozliczenia,id,sprzedaz_dzisiaj_kurczak,sprzedaz_dzisiaj_surówka,temperature,snow,rain,week_day,-1_sprzedaz_dzisiaj_kurczak,-2_sprzedaz_dzisiaj_kurczak,...,-21_sprzedaz_dzisiaj_surówka,-22_sprzedaz_dzisiaj_surówka,-23_sprzedaz_dzisiaj_surówka,-24_sprzedaz_dzisiaj_surówka,-25_sprzedaz_dzisiaj_surówka,-26_sprzedaz_dzisiaj_surówka,-27_sprzedaz_dzisiaj_surówka,-28_sprzedaz_dzisiaj_surówka,-29_sprzedaz_dzisiaj_surówka,-30_sprzedaz_dzisiaj_surówka
0,2021-11-03,1,102.0,90.0,9.866667,0.0,0.000000,2,77.0,0.0,...,72.0,45.0,35.0,0.0,106.0,60.0,90.0,70.0,60.0,52.0
1,2021-11-03,3,59.0,28.0,10.000000,0.0,0.000000,2,51.0,0.0,...,48.0,47.0,36.0,0.0,73.0,66.0,40.0,45.0,48.0,38.0
2,2021-11-03,5,33.0,22.0,9.750000,0.0,0.012500,2,0.0,0.0,...,21.0,24.0,16.0,0.0,48.0,16.0,26.0,25.0,25.0,25.0
3,2021-11-03,6,73.0,54.0,9.733333,0.0,0.000000,2,0.0,0.0,...,78.0,33.0,14.0,0.0,47.0,19.0,46.0,84.0,30.0,22.0
4,2021-11-03,4,44.5,11.0,9.733333,0.0,0.000000,2,36.0,0.0,...,18.0,29.0,18.0,0.0,57.0,27.0,25.0,29.0,35.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,2022-11-17,4,55.5,23.0,3.000000,0.0,0.271333,3,63.0,56.0,...,32.0,28.0,23.0,16.0,0.0,41.0,31.0,21.0,21.0,18.0
2917,2022-11-17,2,146.0,64.0,3.533333,0.0,0.222667,3,105.0,125.0,...,63.0,52.0,54.0,41.0,35.0,104.0,37.0,71.0,56.0,40.0
2918,2022-11-17,8,97.0,49.0,4.400000,0.0,0.136000,3,75.0,82.0,...,28.0,22.0,41.0,17.0,0.0,45.0,37.0,36.0,43.0,25.0
2919,2022-11-17,7,82.0,51.0,3.800000,0.0,0.228667,3,60.0,59.0,...,58.0,45.0,40.0,31.0,0.0,55.0,22.0,50.0,30.0,46.0


### Mean sale value from last N sale days

In [4]:
mean_predictions = pd.DataFrame()
for k in range(1, K+1):
    pred = pd.DataFrame()
    for point in points_of_sales:
        test_set = RCV.getKthTestSet(k, id=point)
        for index, row in test_set.iterrows():
            pred = pd.concat([pred, SimplePredictors.avg(row.dzien_rozliczenia, 7, row.id)], ignore_index=True)
    pred['k'] = k
    mean_predictions = pd.concat([mean_predictions, pred], ignore_index=True)
mean_predictions.to_csv('predictions/mean_predictions.csv')

### Sale value from N days back

In [5]:
n_back_predictions = pd.DataFrame()
for k in range(1, K+1):
    pred = pd.DataFrame()
    for point in points_of_sales:
        test_set = RCV.getKthTestSet(k, id=point)
        for index, row in test_set.iterrows():
            pred = pd.concat([pred, SimplePredictors.back_n_days(row.dzien_rozliczenia, 7, row.id)], ignore_index=True)
    pred['k'] = k
    n_back_predictions = pd.concat([n_back_predictions, pred], ignore_index=True)
n_back_predictions.to_csv('predictions/n_back_predictions.csv')

### Mean value from same days in last N weeks

In [6]:
n_weeks_predictions = pd.DataFrame()
for k in range(1, K+1):
    pred = pd.DataFrame()
    for point in points_of_sales:
        test_set = RCV.getKthTestSet(k, id=point)
        for index, row in test_set.iterrows():
            pred = pd.concat([pred, SimplePredictors.same_days_last_n_weeks_avg(row.dzien_rozliczenia, 7, row.id)], ignore_index=True)
    pred['k'] = k
    n_weeks_predictions = pd.concat([n_weeks_predictions, pred], ignore_index=True)
n_weeks_predictions.to_csv('predictions/n_weeks_predictions.csv')

### EWMA

In [7]:
ewma_predictions = pd.DataFrame()
for k in range(1, K+1):
    pred = pd.DataFrame()
    for point in points_of_sales:
        test_set = RCV.getKthTestSet(k, id=point)
        for index, row in test_set.iterrows():
            pred = pd.concat([pred, SimplePredictors.EWMA(row.dzien_rozliczenia, 3, row.id)], ignore_index=True)
    pred['k'] = k
    ewma_predictions = pd.concat([ewma_predictions, pred], ignore_index=True)
ewma_predictions.to_csv('predictions/ewma_predictions.csv')

### KNN (for each point new model)

In [8]:
knn_predictions = pd.DataFrame()
for k in range(1, K+1):
    preds = pd.DataFrame()
    for point in points_of_sales:
        train = RCV.getKthTrainSet(k, isML=True, id=point)
        test = RCV.getKthTestSet(k, isML=True, id=point)
        days = test['dzien_rozliczenia'].values
        if len(train) == 0 or len(test) == 0:
            continue
        parameters = Params.get_parameters("knn", k, point)
        pred = KNN.knn(train, test, parameters)
        pred['dzien_rozliczenia'] = days
        preds = pd.concat([preds, pred], ignore_index=True)
    preds['k'] = k
    knn_predictions = pd.concat([knn_predictions, preds], ignore_index=True)
knn_predictions.to_csv('predictions/knn_predictions.csv')

### KNN (all points)

In [9]:
knn_all_predictions = pd.DataFrame()
for k in range(1, K+1):
    train = RCV.getKthTrainSet(k, isML=True)
    test = RCV.getKthTestSet(k, isML=True)
    days = test['dzien_rozliczenia'].values
    parameters = Params.get_parameters("knn", k)
    pred = KNN.knn(train, test, parameters)
    pred['k'] = k
    pred['dzien_rozliczenia'] = days
    knn_all_predictions = pd.concat([knn_all_predictions, pred], ignore_index=True)
knn_all_predictions.to_csv('predictions/knn_all_predictions.csv')

### Random Forest (for each point new model)

In [10]:
rf_predictions = pd.DataFrame()
for k in range(1, K+1):
    preds = pd.DataFrame()
    for point in points_of_sales:
        train = RCV.getKthTrainSet(k, isML=True, id=point)
        test = RCV.getKthTestSet(k, isML=True, id=point)
        days = test['dzien_rozliczenia'].values
        if len(train) == 0 or len(test) == 0:
            continue
        parameters = Params.get_parameters("rf", k, point)
        pred = RandomForest.randForest(train, test, parameters)
        pred['dzien_rozliczenia'] = days
        preds = pd.concat([preds, pred], ignore_index=True)
    preds['k'] = k
    rf_predictions = pd.concat([rf_predictions, preds], ignore_index=True)
rf_predictions.to_csv('predictions/rf_predictions.csv')

### Random Forest (all points)

In [11]:
rf_all_predictions = pd.DataFrame()
for k in range(1, K+1):
    train = RCV.getKthTrainSet(k, isML=True)
    test = RCV.getKthTestSet(k, isML=True)
    days = test['dzien_rozliczenia'].values
    parameters = Params.get_parameters("rf", k)
    pred = RandomForest.randForest(train, test, parameters)
    pred['k'] = k
    pred['dzien_rozliczenia'] = days
    rf_all_predictions = pd.concat([rf_all_predictions, pred], ignore_index=True)
rf_all_predictions.to_csv('predictions/rf_all_predictions.csv')